# 🖼️ Pillow (PIL Fork) in Google Colab — Hands‑on Tutorial
Learn core Pillow operations and generate images from ready‑to‑edit templates. This notebook covers opening/saving, drawing, text, compositing, transforms, filters, GIFs, EXIF, and batch workflows — plus editable templates (poster, social card, watermark, meme, collage).

**How to use**
1) Run the setup cell. 2) Explore the "Basics". 3) Try the Templates at the end and tweak parameters.


## 0) Setup

In [ ]:
#@title Install & Imports { display-mode: "form" }
!pip -q install pillow ipywidgets rich

from PIL import Image, ImageOps, ImageEnhance, ImageFilter, ImageDraw, ImageFont, ImageChops, ImageSequence, ImageColor
import numpy as np, io, os, textwrap, math, json, random, pathlib
from IPython.display import display, HTML
from rich import print

# Try to locate a TrueType font. DejaVu is usually present on Colab.
DEFAULT_FONT = "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"
FONT_FALLBACK = ImageFont.load_default()

def get_font(size=40):
    try:
        return ImageFont.truetype(DEFAULT_FONT, size=size)
    except Exception:
        return FONT_FALLBACK

print("[green]Pillow version:[/green]", Image.__version__)
print("[green]Default font present?[/green]", os.path.exists(DEFAULT_FONT))

## 1) Working with files

In [ ]:
# Option A: Upload files via the UI sidebar or programmatically:
from google.colab import files  # type: ignore
# files.upload()  # Uncomment to open the upload dialog

# Option B: Mount Google Drive to persist outputs
# from google.colab import drive  # type: ignore
# drive.mount('/content/drive')
# OUTPUT_DIR = "/content/drive/MyDrive/PillowOutputs"
OUTPUT_DIR = "/content/PillowOutputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)
print("Saving outputs to:", OUTPUT_DIR)

## 2) Create a sample image

In [ ]:
# Let's create a gradient sample image to experiment with
W,H = 800, 500
grad = Image.new('RGB', (W,H), 'black')
px = grad.load()
for y in range(H):
    for x in range(W):
        r = int(255 * x / (W-1))
        g = int(255 * y / (H-1))
        b = int(255 * (1 - x/(W-1)))
        px[x,y] = (r,g,b)
grad.save(f"{OUTPUT_DIR}/sample_gradient.jpg", quality=95)
display(grad)

## 3) Open / Save / Convert

In [ ]:
img = Image.open(f"{OUTPUT_DIR}/sample_gradient.jpg")
print(img.mode, img.size, img.format)

# Convert color modes
rgba = img.convert('RGBA')
luma = img.convert('L')  # grayscale

# Save to different formats
rgba.save(f"{OUTPUT_DIR}/sample_rgba.png")
luma.save(f"{OUTPUT_DIR}/sample_gray.png")
print("Saved:", os.listdir(OUTPUT_DIR))

## 4) Resize / Crop / Pad / Thumbnail

In [ ]:
img = Image.open(f"{OUTPUT_DIR}/sample_gradient.jpg")

# Resize (keep aspect with thumbnail)
thumb = img.copy()
thumb.thumbnail((320, 320))
thumb.save(f"{OUTPUT_DIR}/thumb_320.jpg", quality=92)

# Exact resize ignoring aspect
resized = img.resize((400, 300), Image.BICUBIC)

# Center crop
crop_box = (100, 60, 700, 460)
cropped = img.crop(crop_box)

# Pad to a square
padded = ImageOps.pad(img, (600,600), color='#111')

display(thumb, resized, cropped, padded)

## 5) Rotate / Flip / Affine / Perspective

In [ ]:
img = Image.open(f"{OUTPUT_DIR}/sample_gradient.jpg")

rot = img.rotate(20, expand=True)
flip_h = ImageOps.mirror(img)
flip_v = ImageOps.flip(img)

# Affine transform: scale + shear + translate
aff = img.transform(
    (img.width, img.height),
    Image.AFFINE,
    data=(1.0, 0.2, 0, 0.1, 1.0, 0),
    resample=Image.BICUBIC,
)

display(rot, flip_h, flip_v, aff)

## 6) Filters / Enhance

In [ ]:
img = Image.open(f"{OUTPUT_DIR}/sample_gradient.jpg")

blur = img.filter(ImageFilter.GaussianBlur(3))
edges = img.filter(ImageFilter.FIND_EDGES)
sharper = ImageEnhance.Sharpness(img).enhance(2.0)
contrast = ImageEnhance.Contrast(img).enhance(1.4)
colorboost = ImageEnhance.Color(img).enhance(1.6)

display(blur, edges, sharper, contrast, colorboost)

## 7) Draw primitives & text

In [ ]:
W,H = 900, 500
canvas = Image.new('RGBA', (W,H), (20,20,24,255))
draw = ImageDraw.Draw(canvas)

# Shapes
draw.rectangle((30,30,300,200), outline='#6EE7F9', width=4)
draw.ellipse((350,40,520,210), outline='#FDE68A', width=6)
draw.line((40,260,860,260), fill='#FCA5A5', width=6)
for i in range(10):
    draw.polygon([(600+i*3, 100+i*4), (700+i*2, 180+i*3), (650, 220+i*4)], outline='#C4B5FD')

# Text
font_title = get_font(48)
font_body = get_font(28)
draw.text((40, 280), 'Hello, Pillow!', fill='white', font=font_title)
draw.text((40, 340), 'Draw shapes, lines, and styled text.', fill='#A7F3D0', font=font_body)

canvas.save(f"{OUTPUT_DIR}/draw_demo.png")
display(canvas)

## 8) Alpha compositing & masks

In [ ]:
bg = Image.open(f"{OUTPUT_DIR}/sample_gradient.jpg").convert('RGBA')

# Make a circular mask
mask = Image.new('L', bg.size, 0)
draw = ImageDraw.Draw(mask)
draw.ellipse((150, 60, 650, 460), fill=255)

# A foreground color layer
fg = Image.new('RGBA', bg.size, (255, 255, 255, 0))
draw = ImageDraw.Draw(fg)
draw.rectangle((120, 30, 680, 480), outline=(255,255,255,255), width=10)

# Composite: keep circle of bg + white frame
cut = Image.composite(bg, Image.new('RGBA', bg.size, (0,0,0,0)), mask)
combo = Image.alpha_composite(cut, fg)

combo.save(f"{OUTPUT_DIR}/composite.png")
display(combo)

## 9) Channel operations, blends, and color

In [ ]:
img = Image.open(f"{OUTPUT_DIR}/sample_gradient.jpg").convert('RGBA')
r,g,b,a = img.split()

# Merge channels with a different order to visualize
rgb_swap = Image.merge('RGBA', (b,g,r,a))

# Blend two images
img2 = ImageOps.colorize(img.convert('L'), black='#000022', white='#88FFEE').convert('RGBA')
blended = Image.blend(img, img2, alpha=0.4)

# Color utilities
solid = Image.new('RGBA', img.size, ImageColor.getrgb('salmon') + (180,))
overlayed = Image.alpha_composite(img, solid)

display(rgb_swap, blended, overlayed)

## 10) EXIF basics (if present)

In [ ]:
# EXIF is usually present on photos from cameras/phones; our synthetic image has none.
# Example usage shown for reference:
try:
    sample = Image.open(f"{OUTPUT_DIR}/sample_gradient.jpg")
    exif = sample.getexif()
    print('EXIF tags:', len(exif))
    for tag_id, value in list(exif.items())[:10]:
        print(tag_id, value)
except Exception as e:
    print('EXIF read error:', e)

## 11) Animated GIFs

In [ ]:
# Build a simple animated GIF from generated frames
frames = []
for i in range(12):
    frame = Image.new('RGBA', (360, 200), (30,30,35,255))
    draw = ImageDraw.Draw(frame)
    draw.rectangle((20+i*5,40,160+i*5,160), outline='#A7F3D0', width=6)
    draw.text((10,10), f'Frame {i+1}', fill='white', font=get_font(24))
    frames.append(frame.convert('P', palette=Image.ADAPTIVE))

gif_path = f"{OUTPUT_DIR}/demo_anim.gif"
frames[0].save(gif_path, save_all=True, append_images=frames[1:], duration=120, loop=0, optimize=True)
display(Image.open(gif_path))

## 12) Batch processing example

In [ ]:
# Make a few color variants in batch and save to OUTPUT_DIR
img = Image.open(f"{OUTPUT_DIR}/sample_gradient.jpg")
variants = []
for i, sat in enumerate([0.6, 0.9, 1.2, 1.5]):
    out = ImageEnhance.Color(img).enhance(sat)
    out_path = f"{OUTPUT_DIR}/variant_sat_{sat}.jpg"
    out.save(out_path, quality=92)
    variants.append(out)

# Show a quick grid preview
rows = 1
grid = Image.new('RGB', (img.width*len(variants), img.height))
for i, im in enumerate(variants):
    grid.paste(im, (i*img.width, 0))
display(grid)

---
# 📐 Templates you can edit
Below are ready‑to‑use generators. Change the parameters and run to create your own images.


## Template A: Poster / Flyer

In [ ]:
def make_poster(title='Pillow Workshop', subtitle='Learn image magic in Python', w=1080, h=1350,
                bg_color='#0f172a', accent='#22d3ee', text_color='#ffffff', out_name='poster.png'):
    img = Image.new('RGB', (w,h), bg_color)
    draw = ImageDraw.Draw(img)
    # decorative bars
    draw.rectangle((0,0,w,40), fill=accent)
    draw.rectangle((0,h-40,w,h), fill=accent)
    # title
    title_font = get_font(int(w*0.1))
    subtitle_font = get_font(int(w*0.045))
    tw, th = draw.textbbox((0,0), title, font=title_font)[2:]
    draw.text(((w-tw)//2, int(h*0.28)), title, fill=text_color, font=title_font)
    # subtitle wrapped
    wrap = textwrap.fill(subtitle, width=28)
    draw.multiline_text((int(w*0.08), int(h*0.48)), wrap, fill='#cbd5e1', font=subtitle_font, spacing=8)
    # footer tag
    foot = 'powered by Pillow'
    foot_font = get_font(int(w*0.035))
    fw, fh = draw.textbbox((0,0), foot, font=foot_font)[2:]
    draw.text((w-fw-40, h-fh-28), foot, fill='#94a3b8', font=foot_font)
    out_path = f"{OUTPUT_DIR}/{out_name}"
    img.save(out_path, quality=95)
    display(img)
    print('Saved:', out_path)

make_poster()

## Template B: Social Card (landscape)

In [ ]:
def make_social_card(top_text='New Tutorial', main_text='FFmpeg + Pillow in Colab', 
                      w=1600, h=900, bg='#111827', accent='#38bdf8', out_name='social_card.png'):
    img = Image.new('RGB', (w,h), bg)
    d = ImageDraw.Draw(img)
    # gradient stripe
    stripe = Image.linear_gradient('L').resize((w, int(h*0.45)))
    stripe = ImageOps.colorize(stripe, black='#0ea5e9', white='#22d3ee')
    img.paste(stripe, (0, int(h*0.3)))
    # texts
    top_font = get_font(int(h*0.08))
    main_font = get_font(int(h*0.14))
    d.text((50, 40), top_text, fill='#a5b4fc', font=top_font)
    d.text((50, int(h*0.48)), main_text, fill='#e2e8f0', font=main_font)
    # corner badge
    badge = 'COLAB READY'
    bfont = get_font(int(h*0.05))
    bw, bh = d.textbbox((0,0), badge, font=bfont)[2:]
    pad = 18
    d.rectangle((w-bw-2*pad-40, 40, w-40, 40+bh+2*pad), fill=accent)
    d.text((w-bw-pad-40, 40+pad), badge, fill='#0b1220', font=bfont)
    out_path = f"{OUTPUT_DIR}/{out_name}"
    img.save(out_path, quality=95)
    display(img); print('Saved:', out_path)

make_social_card()

## Template C: Watermark (position & opacity)

In [ ]:
def watermark_image(src_path=f"{OUTPUT_DIR}/sample_gradient.jpg", text='© Your Name',
                    pos='bottom_right', opacity=0.35, out_name='watermarked.png'):
    base = Image.open(src_path).convert('RGBA')
    W,H = base.size
    txt = Image.new('RGBA', base.size, (255,255,255,0))
    d = ImageDraw.Draw(txt)
    font = get_font(int(min(W,H)*0.05))
    tw, th = d.textbbox((0,0), text, font=font)[2:]
    margin = int(min(W,H)*0.04)
    positions = {
        'bottom_right': (W-tw-margin, H-th-margin),
        'bottom_left': (margin, H-th-margin),
        'top_right': (W-tw-margin, margin),
        'top_left': (margin, margin),
        'center': ((W-tw)//2, (H-th)//2)
    }
    xy = positions.get(pos, positions['bottom_right'])
    d.text(xy, text, font=font, fill=(255,255,255,int(255*opacity)))
    out = Image.alpha_composite(base, txt)
    out_path = f"{OUTPUT_DIR}/{out_name}"
    out.convert('RGB').save(out_path, quality=95)
    display(out); print('Saved:', out_path)

watermark_image()

## Template D: Meme Generator (top/bottom text)

In [ ]:
def meme(text_top='WHEN PILLOW', text_bottom='JUST WORKS',
         w=1000, h=800, bg='#222', out_name='meme.png'):
    img = Image.new('RGB', (w,h), bg)
    d = ImageDraw.Draw(img)
    # use white border text
    font = get_font(int(h*0.11))
    def outline_text(x,y, txt):
        for dx,dy in [(-3,0),(3,0),(0,-3),(0,3),(-2,-2),(2,2),(-2,2),(2,-2)]:
            d.text((x+dx,y+dy), txt, font=font, fill='black')
        d.text((x,y), txt, font=font, fill='white')

    # Top centered
    tw, th = d.textbbox((0,0), text_top, font=font)[2:]
    outline_text((w-tw)//2, 20, text_top)
    # Bottom centered
    bw, bh = d.textbbox((0,0), text_bottom, font=font)[2:]
    outline_text((w-bw)//2, h-bh-20, text_bottom)
    out_path = f"{OUTPUT_DIR}/{out_name}"
    img.save(out_path, quality=95)
    display(img); print('Saved:', out_path)

meme()

## Template E: Collage (grid)

In [ ]:
def collage(images=None, cols=3, tile_size=(360,360), gap=8, bg='#0b1220', out_name='collage.png'):
    if images is None:
        # generate sample tiles
        images = []
        for i in range(7):
            im = Image.new('RGB', tile_size, (20+20*i, 50+10*i, 90+10*i))
            d = ImageDraw.Draw(im)
            d.text((10,10), f'Tile {i+1}', fill='white', font=get_font(28))
            images.append(im)
    rows = math.ceil(len(images)/cols)
    W = cols*tile_size[0] + (cols+1)*gap
    H = rows*tile_size[1] + (rows+1)*gap
    canvas = Image.new('RGB', (W,H), bg)
    for idx, im in enumerate(images):
        r, c = divmod(idx, cols)
        thumb = ImageOps.fit(im, tile_size, Image.LANCZOS)
        x = gap + c*(tile_size[0]+gap)
        y = gap + r*(tile_size[1]+gap)
        canvas.paste(thumb, (x,y))
    out_path = f"{OUTPUT_DIR}/{out_name}"
    canvas.save(out_path, quality=95)
    display(canvas); print('Saved:', out_path)

collage()

---
## Tips & Troubleshooting
- If fonts look odd, try another `.ttf` by uploading it and pointing `ImageFont.truetype` to your file.
- Use `OUTPUT_DIR` to control where files go. Mount Drive to persist results.
- Animated GIFs: keep frame count & size modest for speed.
- For transparency, prefer PNG; for photos, prefer JPEG; for vector‑like shapes/flat colors, PNG or WebP lossless.
